### Задачи к Лекции 4

__Исходные данные__ 

Дан файл **"mlbootcamp5_train.csv"**. В нем содержатся данные об опросе 70000 пациентов с целью определения наличия заболеваний сердечно-сосудистой системы (ССЗ). Данные в файле промаркированы и если у человека имееются ССЗ, то значение **cardio** будет равно 1, в противном случае - 0. Описание и значения полей представлены во второй лекции.

__Загрузка файла__

In [28]:
%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from matplotlib import pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [10, 5]


df = pd.read_csv("D://шкИла//третекурс//6 сем//обработка речи//mlbootcamp5_train.csv", 
                 sep=";", 
                 index_col="id")
# Делаем one-hot кодирование
chol = pd.get_dummies(df["cholesterol"], prefix="chol")
gluc = pd.get_dummies(df["gluc"], prefix="gluc")
df = pd.concat([df, chol, gluc], axis=1)

# Делаем пол бинарным признаком
df["gender_bin"] = df["gender"].map({1: 0, 2: 1})
df.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,chol_1,chol_2,chol_3,gluc_1,gluc_2,gluc_3,gender_bin
id,,,,,,,,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0,True,False,False,True,False,False,1
1,20228,1,156,85.0,140,90,3,1,0,0,1,1,False,False,True,True,False,False,0
2,18857,1,165,64.0,130,70,3,1,0,0,0,1,False,False,True,True,False,False,0
3,17623,2,169,82.0,150,100,1,1,0,0,1,1,True,False,False,True,False,False,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0,True,False,False,True,False,False,0


## Задачи

__1. Хоть в sklearn и присутствует реализация метода k-ближайших соседей, я же предлагаю попробовать вам написать его самостоятельно.__

* __создать классификатор используя только pandas, numpy и scipy. Гиперпараметром данного классификатора должно быть число ближайших соседей. (Необязательно) можно добавить метрику расстояния и выбор весов.__
* __С помощью кросс-валидации найти оптимальное количество ближайших соседей и (необязательно) набор признаков.__

Алгоритм работы классификатора:
 1. Для заданного прецедент  $\vec{x}$ мы считаем расстояние до всех прецедентов в обучающей выборке.
 2. Сортируем прецеденты по расстоянию до $\vec{x}$.
 3. Отбираем $k$ минимальных значений
 4. Устраиваем голосование между отобранными прецедент.

In [29]:
data = df[(df["ap_hi"] >= 50) & (df["ap_hi"] <= 200)]
data = data[(df["ap_lo"] >= 50) & (data["ap_lo"] <= 200)]
data = data[(data['height'] > 100) & (data['height'] < 250)]
data = data[(data['weight'] > 30) & (data['weight'] < 200)]
data = data[data['ap_lo'] < data['ap_hi']]

In [30]:
from collections import Counter

class KNNClassifier:
    def __init__(self, k=3):
        self.k = k
        self.X_train = None
        self.y_train = None

    def fit(self, X, y):
        self.X_train = X.to_numpy()
        self.y_train = y.to_numpy() #обучение и сохранение в виде нумпай объекта

    def predict(self, X):
        X = X.to_numpy()
        predictions = []
        for x in X: #х - строка из Х
            x = x.astype(float)  #а то там ошибка была
            #евклидово расстояние до объектов из трейна
            distances = np.sqrt(np.sum((self.X_train - x) ** 2, axis=1))
            #значения ближайших соседей
            k_indices = np.argsort(distances)[:self.k]
            #значения ближайших соседей
            k_neighbor_labels = self.y_train[k_indices]
            #выбираем наиболее частый класс
            most_common = Counter(k_neighbor_labels).most_common(1)[0][0]
            predictions.append(most_common)
        return np.array(predictions)

In [31]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

#поиск k
def cross_validate_knn(X, y, k_values, n_splits=5):
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42) #фолд - одна часть (здесь делю на 5)
    best_k = None
    best_score = 0
    scores = {}

    for k in k_values:
        accuracies = []
        for train_idx, test_idx in kf.split(X):
            X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
            y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
            
            model = KNNClassifier(k=k) #обучаем с текущим k
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            acc = accuracy_score(y_test, y_pred)
            accuracies.append(acc)
        
        avg_acc = np.mean(accuracies) #средняя точность по фолду
        scores[k] = avg_acc

        if avg_acc > best_score:
            best_score = avg_acc
            best_k = k

    return best_k, best_score, scores

In [34]:
features = ['age', 'height', 'weight', 'ap_hi', 'ap_lo',
            'chol_1', 'chol_2', 'chol_3',
            'gluc_1', 'gluc_2', 'gluc_3',
            'gender_bin']

X = data[features]
y = data['cardio']

In [35]:
print(X.shape)

(68559, 12)


In [ ]:
k_values = range(1, 16)
X = X.astype(float)
best_k, best_score, scores = cross_validate_knn(X, y, k_values)

print(f"Best k: {best_k}, Accuracy: {best_score:.4f}")

In [ ]:
final_model = KNNClassifier(k=best_k)
final_model.fit(X, y)

In [ ]:
y_pred = final_model.predict(X_test)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
final_model = KNNClassifier(k=best_k)
final_model.fit(X_train, y_train

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

y_pred = final_model.predict(X_test)

# Метрики
print("Accuracy:", accuracy_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_pred))

**Комментарии:** Ваши комментарии здесь.

**2. Определить какой из трех классификаторов (kNN, наивный Байес, решающее дерево) лучший в каждой метрике по отдельности: accuracy, F1-мера, ROC AUC, функция потерь. Использовать набор признаков: 'age', 'weight', 'height', 'ap_lo', 'ap_hi'.**

**(Необязательно) Найти оптимальный набор признаков.**

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

features = ["age", "weight", "height", "ap_lo", "ap_hi"]
X = df[features]
y = df["cardio"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

#масштаирование
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

models = {
    "kNN": KNeighborsClassifier(n_neighbors=5),
    "Naive Bayes": GaussianNB(),
    "Decision Tree": DecisionTreeClassifier(random_state=42)
}

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, log_loss

results = {}

for name, model in models.items():
    #обучение
    if name == "kNN":
        model.fit(X_train_scaled, y_train)
        y_pred = model.predict(X_test_scaled)
        y_proba = model.predict_proba(X_test_scaled)
    else:
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)

    #метрики
    results[name] = {
        "Accuracy": accuracy_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred),
        "ROC AUC": roc_auc_score(y_test, y_proba[:, 1]),
        "Log Loss": log_loss(y_test, y_proba)
    }

In [ ]:
import pandas as pd

results_df = pd.DataFrame(results).T
print(results_df)

for metric in results_df.columns:
    best_model = results_df[metric].idxmax() if metric != "Log Loss" else results_df[metric].idxmin()
    print(f"по метрике {metric}: {best_model} - лучший")

**Комментарии:** Ваши комментарии здесь.